In [ ]:
from joblib import Parallel, delayed
from neuropy.utils.signal_process import filter_sig

sessions = subjects.ripple_sess()

get_extrema = lambda arr: arr[np.argmax(np.abs(arr))]


def get_max_val(x, t, bins):
    return stats.binned_statistic(t, x, bins=bins, statistic=get_extrema)[0][::2]


for sub, sess in enumerate(tqdm(sessions)):

    srate= sess.eegfile.sampling_rate
    good_channels = np.concatenate(sess.probegroup.get_connected_channels()).astype(
        "int"
    )
    rpl_epochs = sess.ripple.flatten()
    rpl_starts = rpl_epochs[::2]
    rpl_traces, t = sess.eegfile.get_frames_within_epochs(
        sess.ripple, good_channels, ret_time=True
    )
    rpl_traces = filter_sig.bandpass(rpl_traces, lf=2, hf=30, fs=srate)

    max_val = Parallel(n_jobs=8)(
        delayed(get_max_val)(arr, t, rpl_epochs) for arr in rpl_traces
    )
    max_val = np.asarray(max_val)

    # raw amplitude in mV
    amp_raw = np.ptp(max_val, axis=0) * 0.95 * 1e-3

    new_epochs = sess.ripple.add_column("sharp_wave_amp", amp_raw)
    new_epochs.save(sess.filePrefix.with_suffix(".ripple"))
